# Modelo de Pasillo de Compra 🛒🛍️
Este proyecto busca optimizar la experiencia de compra en retail mediante la predicción de la próxima línea de productos que un cliente puede adquirir, basándose en su historial de compras y patrones de consumo de otros usuarios. El sistema ofrece recomendaciones personalizadas, generalizadas y de recompra para clientes conocidos, y también proporciona recomendaciones iniciales para clientes nuevos.



In [2]:
import pandas as pd
import yaml,joblib,datetime
from transforms import Pipeline_data
from model.Model_Xgboost import Pipeline_data_modelCD,model_training_testing
from model.Model_Apriori import Learning_paths
from transforms import cleaningData
import sys

%load_ext autoreload
%autoreload 2

with open('configPasillo.yml', 'r',encoding='utf-8') as file:
    config_data = yaml.safe_load(file)

### Descarga de datos

In [ ]:
from data import cargaDatos_train_model
df_recomendaciones = cargaDatos_train_model.Carga_tipo_datos('Data para entrenamiento',query_text=config_data['Query_recomendaciones'],filtro='',env='prd',verbose=False)

### XGboost

#### Entrenamiento

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="mlflow")

df_train,params_context = Pipeline_data.Preparar_data_paraTrain(df)
df_ult,df_ant = Pipeline_data_modelCD.pipelinePrepare_data_train(df_train,'prd',transformacion='total')
Diccionario_entrenable =  Pipeline_data_modelCD.Rebalancear_y_organizarDfs(y=df_ult,X=df_ant) 
Resultados_de_modelos = model_training_testing.Entrenar_modelos_por_clase(Diccionario_entrenable=Diccionario_entrenable,params_context=params_context,nuevo_umbral=0.5,versionar=True)

In [ ]:
from model.MlFlow import Pipeline_mlflow,Mlflow_deploy
import yaml
from model.Model_Xgboost import Pipeline_data_modelCD,model_training_testing

#Descarga de modelos personalizados
#models_mlflow_disponibilizados,input_schema,df_runs_best_models = Pipeline_mlflow.Pipeline_modelos_disponibles_MLFLOW(tags=config_data['tipo_de_modelo_rec_pers'],metrica_seleccionada = config_data['Metrica_seleccion_Models_Mlflow'])
models_mlflow_disponibilizados,input_schema,df_runs_best_models = Pipeline_mlflow.UtilizarModelosEntrenados(tags=config_data['tipo_de_modelo_rec_pers'],metrica_seleccionada=config_data['Metrica_seleccion_Models_Mlflow'])
y_test_global,y_pred_global,metricas = Pipeline_data_modelCD.Obtener_predicciones_testTotal(Resultados_de_modelos=Resultados_de_modelos,models_mlflow_disponibilizados=models_mlflow_disponibilizados
                                                                          ,input_schema=input_schema,config_data=config_data)
    
Mlflow_deploy.Almacenar_Artefacto_MlFlow(artefacto=(models_mlflow_disponibilizados,input_schema,df_runs_best_models),config_data=config_data,metricas=metricas)

#display(pd.DataFrame(model_training_testing.Metricas_globales(Resultados_de_modelos),index=config_data['metricas_names']).T)
#model_training_testing.Revisar_Metricas_individuales(Resultados_de_modelos)

Obtener modelos de XGBoost

#### Inferencia

In [ ]:
from model.Model_Xgboost import Recom_XLvl
from transforms import cleaningData
from model.MlFlow import Pipeline_mlflow

models_mlflow_disponibilizados,input_schema,df_runs_best_models_gen = Pipeline_mlflow.UtilizarModelosEntrenados(tags=config_data['tipo_de_modelo_rec_pers_global'],metrica_seleccionada=config_data['Metrica_seleccion_Models_Mlflow'],update=False)[0]['xgboost_stack.pkl']

#clientes = cleaningData.obtenerVector_ultimaCompra_Client(df=df,nivel='UNIDAD_FECHA',maximo_compras=3)
#infoClientes= Pipeline_data.Obtener_informaciondeComprasAnteriores(df=clientes,config_data=config_data,env='prd')
clientes,clientes_index,df_vectores = Recom_XLvl.Organizar_df_para_predict_xgboost(df=clientes,input_schema=input_schema)
Recomendaciones_xgboost = Recom_XLvl.Recomendar_xgboost_lvl(Models=models_mlflow_disponibilizados,input_schema=input_schema,data=clientes,
                                                            n_cliente=clientes_index,config_data=config_data,df_vector_compras=df_vectores)

### A priori

#### Entrenamiento de modelo

In [113]:
with open('configPasillo.yml', 'r',encoding='utf-8') as file:
    config_data = yaml.safe_load(file)

df_train,params_context = Pipeline_data.Preparar_data_paraTrain(df)
df_apriori,records = Learning_paths.pipelineData_Apriori(df_train,config_data['lvl_apriori'])
association_results =  Learning_paths.Apriori_Asociaciones_MlFlow(records=records,params_context=params_context,min_support=config_data['min_support'],
                                                                  min_confidence=config_data['min_confidence'],min_lift=config_data['min_lift'],
                                                                  max_length=config_data['max_length'],versionar=True)

2024/11/24 18:12:12 INFO mlflow.tracking.fluent: Experiment with name 'Apriori_rules' does not exist. Creating a new experiment.
2024/11/24 18:12:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run respected-donkey-465 at: http://localhost:5000/#/experiments/328871494425630856/runs/f987ad2c4eb945b99e20da9964dbf345.
2024/11/24 18:12:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/328871494425630856.


Derivamos 117 reglas generales, y 168 combinaciones de asociación en run->-> f987ad2c4eb945b99e20da9964dbf345.


#### Inferencia

In [8]:
from model.Model_Apriori import Learning_paths,Recom_Lvl

from model.MlFlow import Pipeline_mlflow
import yaml
with open('configPasillo.yml', 'r') as file:
    config_data = yaml.safe_load(file)
    
#Descarga de modelo generalizado
models_mlflow_disponibilizados_gen,input_schema_apriori,df_runs_best_models_gen = Pipeline_mlflow.UtilizarModelosEntrenados(tags=config_data['tipo_de_modelo_rec_genr'],metrica_seleccionada=config_data['Metrica_seleccion_Models_Apriori'],run=config_data['run_apriori'])
#models_mlflow_disponibilizados_gen,input_schema_apriori,df_runs_best_models_gen = Pipeline_mlflow.UtilizarModelosEntrenados(tags=config_data['tipo_de_modelo_rec_genr'])
Reglas_pd = Learning_paths.Mostrar_reglas_de_asociacion(models_mlflow_disponibilizados_gen['Apriori.pkl'])
clientes = cleaningData.obtenerVector_ultimaCompra_Client(df=df,nivel='UNIDAD_FECHA',maximo_compras=3)
recomendaciones_apriori = Recom_Lvl.Generar_recomendaciones_sublineas(clientes=clientes,Reglas_pd=Reglas_pd,config_data=config_data,MostrarAntesEliminar=None)
#recomendaciones_apriori.head(2)

Actualmente hay 117 reglas generales de asociación
Actualmente hay 168 combinaciones de asociación


Limpiando Recomendaciones: 100%|██████████| 3/3 [12:06<00:00, 242.25s/it]


Se han eliminado -1562602 registros de recomendaciones -> ya lo tiene el cliente en forma exacta
Se han eliminado -1082522 registros de recomendaciones -> ya lo tiene el cliente en forma similar
Se han eliminado -1545092 registros de recomendaciones -> ´Tiene recomendaciones duplicadas
La forma de las predicciones es (4856283, 9)
Forma antes de reducir predicciones (4856283, 9)
Forma despues de reducir las predicciones (2334986, 10)


### Recomendación de productos

#### Por disimilaridad

In [ ]:
from model.src_recProduct import Prods_atrBased
from transforms import Pipeline_data

df_recomendaciones_Atr= Pipeline_data.AgregarInfoCliente(df_recomendaciones=Preds_Clientes,infoClientes=infoClientes)
df_recomendaciones_Atr,df_prods = Prods_atrBased.Complementar_Recomendaciones(df_recomendaciones_Atr,env='prd',config_data=config_data,cant_prods=9999,generacion={3},MostrarAntesEliminar=None)

### Prueba Cliente 

In [178]:
import requests
# URL de la API
api_url = "http://127.0.0.1:8000"

params = {"cliente": "1140839719"}

response = requests.get(api_url, params=params)
# Comprobamos el código de estado de la respuesta
if response.status_code == 200:
    # Convertimos la respuesta en JSON
    data = response.json()
data

{'historial_compras': {'id_cliente': '1140839719',
  'ultima_compra': 'SOCMBI,SOSASR,DODBCDB,SOCMC6PTO,SOSAMC',
  'skus_compras': '7016520;7008713;7009058;7009743;7010500',
  'atributos': 'ARLES BLANCO ESPEJO GLAM SOCARLES SOCBLANCO SOCESPEJO SOCGLAM MONET LINO LENON PERLA TAUPE BEIGE TELA GLAM PERLA SOCMONET SOCLINO SOCLENON SOCPERLA SOCTAUPE SOCBEIGE SOCTELA SOCGLAM SOCPERLA FULL ARL',
  'rango_desembolso': 3.0,
  'Cupo_dispo': 10000000.0,
  'Ultima_medida': 'FULL'},
 'recomendaciones': [{'sublinea': 'COF',
   'confianza': 0.656,
   'productos': '7032288;7032827;7034409',
   'prods_cupo': 'JA_7032288;JA_7032827;JA_7034409',
   'similitud': '0.0;0.0;0.0',
   'codigo_bono': 'AIL032QV9WIHIUQ8SWD399ZR54R1WU23A28HE2'},
  {'sublinea': 'NO',
   'confianza': 0.365,
   'productos': '7031989;7019304;7033200',
   'prods_cupo': 'JA_7031989;JA_7019304;JA_7033200',
   'similitud': '0.4346;0.4346;0.4346',
   'codigo_bono': 'W5GZ4PERNT6BNUCFMR9ABU70FE73B19TH34TFJ'},
  {'sublinea': 'BU',
   'confianz